In [ ]:
# Importing all the necessary libraries
import sys
import os
from PIL import Image
import time

import tkinter as tk
from tkinter import ttk
from tkinter import filedialog as fd
import customtkinter

from matplotlib.backends.backend_tkagg import (
    FigureCanvasTkAgg, NavigationToolbar2Tk)  # type: ignore
from matplotlib.figure import Figure
from matplotlib import pyplot as plt
import matplotlib.animation as animation
from matplotlib import style
from matplotlib.widgets import SpanSelector
from matplotlib.backend_bases import key_press_handler

import numpy as np

import random
import serial
import serial.tools.list_ports as list_ports
import json

import cv2


In [ ]:
# Modes: "System" (standard), "Dark", "Light"
from typing import Optional, Tuple, Union

# Modes: "System" (standard), "Dark", "Light"
customtkinter.set_appearance_mode("Dark")
# Themes: "blue" (standard), "green", "dark-blue"
customtkinter.set_default_color_theme("blue")
# customtkinter.deactivate_automatic_dpi_awareness()


class MenuBar(customtkinter.CTkFrame):
    def __init__(self, master):
        super().__init__(master)


class Logo(customtkinter.CTkFrame):
    def __init__(self, master):
        super().__init__(master)
        self.menu_logo_image = customtkinter.CTkImage(Image.open(
            os.path.join(master.image_path, "lst_logo.png")), size=(master.winfo_width(), 53*master.winfo_width()/131))
        self.menu_logo = customtkinter.CTkLabel(
            self, image=self.menu_logo_image, text='')
        self.menu_logo.pack(fill="both", expand=True)
        # self.menu_logo.bind('<Configure>',_resize_image)

def _resize_image(self,event):
    new_width = event.width
    new_height = event.height
    pilimg = self.pirates.resize((new_width,new_height)) 
    self.homelbl.configure(image=customtkinter.CTkImage(pilimg))


class SerialConsole(customtkinter.CTkFrame):
    def __init__(self, master):
        super().__init__(master)


class CameraView(customtkinter.CTkFrame):
    def __init__(self, master):
        super().__init__(master)


class ScanParameters(customtkinter.CTkFrame):
    def __init__(self, master):
        super().__init__(master)


class CNCButtons(customtkinter.CTkFrame):
    def __init__(self, master):
        super().__init__(master)


class RamanMapSpectrum(customtkinter.CTkFrame):
    def __init__(self, master):
        super().__init__(master)


class CNCStatus(customtkinter.CTkFrame):
    def __init__(self, master):
        super().__init__(master)


class SettingsWindow(customtkinter.CTk):
    def __init__(self, master):
        super().__init__(master)


class QuitAppWindow(customtkinter.CTkToplevel):
    def __init__(self, master):
        super().__init__(master)

        self.geometry(
            f"320x100+{App.winfo_x()+App.winfo_width()//2-160}+{App.winfo_y()+App.winfo_height()//2-50}")
        self.title("Quit Application")
        self.resizable(False, False)
        self.deiconify()
        self.grab_set()

        label = customtkinter.CTkLabel(
            self, text="Are you sure you want to quit?")
        label.grid(row=0, column=0, columnspan=2, padx=10, pady=10)
        yes_button = customtkinter.CTkButton(
            self, text="Yes", command=App._quit)
        yes_button.grid(row=1, column=0, padx=10, pady=10)
        no_button = customtkinter.CTkButton(
            self, text="No", command=self.destroy)
        no_button.grid(row=1, column=1, padx=10, pady=10)


class App(customtkinter.CTk):
    def __init__(self):
        super().__init__()
        self.protocol("WM_DELETE_WINDOW", self.OnQuitApp)
        self.title("LST Raman Microscope Scanner")
        screen_width = self.winfo_screenwidth()
        screen_height = self.winfo_screenheight()
        app_window_width = 1280
        app_window_height = 720
        self.geometry(
            f"{app_window_width}x{app_window_height}+{int(screen_width/2-app_window_width/2)}+{int(screen_height/2-app_window_height/2)}")
        self.minsize(app_window_width, app_window_height)
        self.grab_set()
        self.initialdir = os.getcwd()
        self.icons_folder = 'icons'

        # current_folder = os.getcwd()
        current_folder = globals()['_dh'][0]
        self.image_path = os.path.join(self.initialdir, self.icons_folder)
        try:
            self.iconbitmap(os.path.join(self.image_path, "lst_logo.ico"))
        except:
            pass

        # set grid layout 2x4
        self.grid_rowconfigure(0, weight=2)
        self.grid_rowconfigure(1, weight=7)
        self.grid_rowconfigure(2, weight=2)
        self.grid_columnconfigure(0, weight=2)
        self.grid_columnconfigure(1, weight=6)
        self.grid_columnconfigure(2, weight=3)
        self.grid_columnconfigure(3, weight=2)

        # create frames
        self.menu_bar_frame = MenuBar(self)
        self.menu_bar_frame.grid(
            row=0, column=0, rowspan=2, padx=(20, 10), pady=(20, 10), sticky="nsew")

        self.logo_frame = Logo(self)
        self.logo_frame.grid(row=2, column=0, padx=(
            20, 10), pady=(10, 20), sticky="nsew")

        self.raman_map_spectrum_frame = RamanMapSpectrum(self)
        self.raman_map_spectrum_frame.grid(
            row=0, column=1, rowspan=3, padx=10, pady=20, sticky="nsew")

        self.camera_view_frame = CameraView(self)
        self.camera_view_frame.grid(
            row=0, column=2, rowspan=2, padx=10, pady=(20, 10), sticky="nsew")

        self.serial_console_frame = SerialConsole(self)
        self.serial_console_frame.grid(
            row=2, column=2, padx=10, pady=(10, 20), sticky="nsew")

        self.cnc_status_frame = CNCStatus(self)
        self.cnc_status_frame.grid(row=0, column=3, padx=(
            10, 20), pady=(20, 10), sticky="nsew")

        self.cnc_buttons_frame = CNCButtons(self)
        self.cnc_buttons_frame.grid(
            row=1, column=3, rowspan=2, padx=(10, 20), pady=(10, 20), sticky="nsew")

    def OnQuitApp(self):
        quit_app_window = customtkinter.CTkToplevel(self)
        quit_app_window.geometry(
            f"320x100+{self.winfo_x()+self.winfo_width()//2-160}+{self.winfo_y()+self.winfo_height()//2-50}")
        quit_app_window.title("Quit Application")
        quit_app_window.resizable(False, False)
        quit_app_window.deiconify()
        quit_app_window.grab_set()

        label = customtkinter.CTkLabel(
            quit_app_window, text="Are you sure you want to quit?")
        label.grid(row=0, column=0, columnspan=2, padx=10, pady=10)
        yes_button = customtkinter.CTkButton(
            quit_app_window, text="Yes", command=self._quit)
        yes_button.grid(row=1, column=0, padx=10, pady=10)
        no_button = customtkinter.CTkButton(
            quit_app_window, text="No", command=quit_app_window.destroy)
        no_button.grid(row=1, column=1, padx=10, pady=10)

    def _quit(self):
        self.quit()
        self.destroy()


if __name__ == "__main__":
    app = App()
    app.mainloop()